In [4]:
import os 
import numpy as np 
import pandas as pd
from collections import defaultdict
import itertools

In [5]:
datadir = '../../data/goodbooks-10k-master/'

In [6]:
ratings_file = datadir + 'ratings.csv'
df = pd.read_csv(ratings_file, delimiter=',')

In [7]:
df.dataframeName = 'ratings.csv'
print(df.columns)

Index(['user_id', 'book_id', 'rating'], dtype='object')


In [8]:
print(df.head(5))

   user_id  book_id  rating
0        1      258       5
1        2     4081       4
2        2      260       5
3        2     9296       5
4        2     2318       3


In [9]:
print (max(df['book_id']), min(df['book_id']))
print (max(df['user_id']), min(df['user_id']))
print (max(df['rating']), min(df['rating']))
print(len(df))

10000 1
53424 1
5 1
5976479


In [10]:
#First create a threshold for the ratings to consider: rating_threshold
#Drop all entry with rating lower than the threshold
rating_threshold = 3.5
df = df[df.rating >= rating_threshold]
print(len(df))
user_id_set = set(df['user_id'])
movie_id_set = set(df['book_id'])
print(len(user_id_set))
print(len(movie_id_set))

4122111
53406
10000


In [ ]:
###Question, can we use the rating below for ex, 2.5 as an negative association?

In [21]:
#dict with key: user, val: the bookID usr rated above given threshold
user_dict = defaultdict(list)
for index, row in df.iterrows():    
    bookId = int(row['book_id'])
    usrId = int(row['user_id'])
    user_dict[usrId].append(bookId)
print(len(user_dict))

53406


In [29]:
#marginal counts included, without marginal counts, the dict len is 20537173
#also, not using usernum threshold here, we do not remove all records about the movie if, it does not have more than 50 user entries
#if a book_book pair count is too little we can delete the entry as well. not yet implemented
book_book_count =  defaultdict(lambda: 0)
for key, val in user_dict.items():
    for i in range(len(val)):
        for j in range(i, len(val)):
            book_book_count[(val[i], val[j]) if val[i] <= val[j] else (val[j], val[i])] += 1  

In [30]:
print(len(book_book_count))

20547173


In [44]:
print(book_book_count[(10,11)])
print(book_book_count[(10.0,11.0)])
print(book_book_count[(10,10)])

3120
3120
10562


In [38]:
maxval = max([v for k,v in book_book_count.items()])
print(maxval)

18926


In [56]:
def marginal_prob(movie_id, count_matrix, num_users):
    '''function to get the marginal prob:
        P(movie_id1)       
    '''
    margn_count = count_matrix[(movie_id, movie_id)]
    return margn_count/num_users


def joint_prob(movie_id1, movie_id2, count_matrix, num_users):
    '''function to get the joint prob:
        P(movie_id1, movie_id2)
    '''
    key = (movie_id1, movie_id2) if movie_id1<= movie_id2 else (movie_id2, movie_id1) 
    joint_count = count_matrix[key] 
    return joint_count/num_users


def conditional_prob(movie_id1, movie_id2, count_matrix):
    '''function to get the conditional prob:
        P(movie_id1 | movie_id2)       
    '''
    key = (movie_id1, movie_id2) if movie_id1<= movie_id2 else (movie_id2, movie_id1)
    joint_count = count_matrix[key]
    margn_count = count_matrix[movie_id2, movie_id2]
    return joint_count/margn_count

In [58]:
num_users = len(user_id_set)
print(num_users)
print(min(user_id_set), max(user_id_set))
print("marginal")
print(marginal_prob(10, book_book_count, num_users))
print(marginal_prob(11, book_book_count, num_users))
print("joint")
print(joint_prob(10, 11, book_book_count, num_users))
print(joint_prob(11, 10, book_book_count, num_users))
print("conditional")
print(conditional_prob(10, 11, book_book_count))
print(conditional_prob(11, 10, book_book_count))
print()

53406
1 53424
marginal
0.19776804104407744
0.1897165112534172
joint
0.058420402201999774
0.058420402201999774
conditional
0.30793525463876825
0.2953985987502367



In [ ]:
#now we have the book_book_pair and the methods to calculate any pair's joint/ conditional
#prob, if we need, we can generate the entire joint/conditional prob matrix as well.

In [ ]:
'''
# Sample example
# Conditioning CAN be less than the marginal! 
# ref: 2018-box-paper table-1

for k in final_dict.keys():
    p1 = conditional_prob(10, k, count_matrix)
    p2 = marginal_prob(10, count_matrix, N)
    if p1 <= p2:
        print(k, p1)
'''

In [88]:
#ToDo
#Filter out tags that does not make sense
#Right now I only delete the entry in the book_tags.csv if the count is less than a threshold
#We can also manuallly remove tags in the tags.csv if we think the tag does not make sense

In [89]:
#build a dict betweer goodread_book_id and the book_id used in this dataset
books = datadir + 'books.csv'
df_books = pd.read_csv(books, delimiter=',')

In [90]:
df_books.dataframeName = 'books.csv'
df_books= df_books[['book_id', 'goodreads_book_id']]
print(df_books.columns)

Index(['book_id', 'goodreads_book_id'], dtype='object')


In [131]:
#dict with key: goodreads_book_id, val: book_id 
book_id_dict = defaultdict(lambda:-1)
for index, row in df_books.iterrows():    
    bookId = int(row['book_id'])
    GoodReadId = int(row['goodreads_book_id'])
    book_id_dict[GoodReadId]=bookId
print(len(book_id_dict))
print(book_id_dict[1])

10000
27


In [126]:
#load book_tags file
book_tags = datadir + 'book_tags.csv'
df_book_tags = pd.read_csv(book_tags, delimiter=',')

In [127]:
df_book_tags.dataframeName = 'book_tags.csv'
print(df_book_tags.columns)

Index(['goodreads_book_id', 'tag_id', 'count'], dtype='object')


In [128]:
print(df_book_tags.head(5))

   goodreads_book_id  tag_id   count
0                  1   30574  167697
1                  1   11305   37174
2                  1   11557   34173
3                  1    8717   12986
4                  1   33114   12716


In [129]:
print (max(df_book_tags['goodreads_book_id']), min(df_book_tags['goodreads_book_id']))
print (max(df_book_tags['tag_id']), min(df_book_tags['tag_id']))
print (max(df_book_tags['count']), min(df_book_tags['count']))
print(len(df_book_tags))
#There are negative counts?

33288638 1
34251 0
596234 -1
999912


In [120]:
#remove entry if tag count is lower than given threshold, in which case the association between the tag and the book is
#not strong
tag_count_threshold = 500
df_book_tags = df_book_tags[df_book_tags['count'] >= tag_count_threshold]
print(len(df_book_tags))

30491


In [130]:
#convert the good_read_book_id to book_id in this dataframe
for index, row in df_book_tags.iterrows():
    bookId = book_id_dict[int(row['goodreads_book_id'])]
    df_book_tags.set_value(index,'goodreads_book_id', bookId) 
print(df_book_tags.head(5))

/home/htann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


   goodreads_book_id  tag_id   count
0                 27   30574  167697
1                 27   11305   37174
2                 27   11557   34173
3                 27    8717   12986
4                 27   33114   12716


In [133]:
# rename the column name from goodread_id to book_id
df_book_tags.rename(columns={'goodreads_book_id': 'book_id'}, inplace=True)
print(df_book_tags.head(5))

   book_id  tag_id   count
0       27   30574  167697
1       27   11305   37174
2       27   11557   34173
3       27    8717   12986
4       27   33114   12716


In [134]:
#dict with key: book_id, val: tag_id, to list all the tags every book has
book_tag_dict = defaultdict(list)
for index, row in df_book_tags.iterrows():    
    bookId = int(row['book_id'])
    tagId = int(row['tag_id'])
    book_tag_dict[bookId].append(tagId)
print(len(book_tag_dict))

10000


In [135]:
#marginal counts included
#here the tag tag pair is incremented by one if a book is listed with both of the two tags
tag_tag_count =  defaultdict(lambda: 0)
for key, val in book_tag_dict.items():
    for i in range(len(val)):
        for j in range(i, len(val)):
            tag_tag_count[(val[i], val[j]) if val[i] <= val[j] else (val[j], val[i])] += 1  

In [136]:
print(len(tag_tag_count))

5890222
